Zadanie 2 Przetwarzanie i Analiza Dużych zbiorów danych
Barbara Morawska 234096
Andrzej Sasinowski 234118
Marcin Markiewicz 234090

In [ ]:
def prepare_data():
    return  sc.textFile("FileStore/shared_uploads/234090@edu.p.lodz.pl/4.txt") \
           .map(lambda line:line.split("\t")) \
           .map(lambda line: (int(line[0]), list(map(lambda x: int(x), line[1].split(',')))))
def get_pairs(line):
    user = line[0]
    friends = line[1]
    friends_pairs = []
    for friend in friends:
        friends_pairs.append(((user, friend),0)) # direct friendships
    for friend1 in friends:
       for friend2 in friends:
           if(friend1 != '' and friend2 != ''):  
                if friend1 != friend2:
                    friends_pairs.append(((friend1, friend2), 1)) #indirect friendships
    return friends_pairs
def get_recommendation(m): 
    return [(m[0][0], (m[0][1], m[1])), (m[0][1], (m[0][0], m[1]))]
def sort_and_slice_recommendation(recs):
    recs.sort(key=lambda x: (-x[1], x[0]))
    return list(map(lambda x: x[0], recs))[:10]
   
def setup():
    data = prepare_data()
    pairs = data.flatMap(get_pairs)
    pairs.cache()
    recommendations =  pairs.groupByKey() \
                              .filter(lambda x: 0 not in x[1]) \
                              .map(lambda x: (x[0], sum(x[1]))) \
                              .flatMap(get_recommendation) \
                              .groupByKey() \
                              .map(lambda m: (m[0], sort_and_slice_recommendation(list(m[1]))))\
                              .sortByKey() 
                             
    output = recommendations.collect()
    output = ''.join(str(list(map(lambda m: (str(m[0])+'\t'+str(m[1])+'\n'),output))))
    dbutils.fs.put("/FileStore/Shared/output1.txt", output)
setup()

Do Stworzenia naszego algorytmu wykorzystaliśmy Apache Spark wraz z językiem Python, do testów wykorzystaliśmy databricks. Posłużyliśmy się starszym API Sparka RDD. Apache Spark rozszerza bibliotekę narzędzi pythona o wiele przydatnych funkcji jak np flatMap czy groupByKey.

Rezultaty dla zmiennych określonych w zadaniu
924 [439, 2409, 6995, 11860, 15416, 43748, 45881]
8941 [8943, 8944, 8940]
8942 [8939, 8940, 8943, 8944]
9019 [9022, 317, 9023]
9020 [9021, 9016, 9017, 9022, 317, 9023]
9021 [9020, 9016, 9017, 9022, 317, 9023]
9022 [9019, 9020, 9021, 317, 9016, 9017, 9023]
9990 [13134, 13478, 13877, 34299, 34485, 34642, 37941]
9992 [9987, 9989, 35667, 9991]
9993 [9991, 13134, 13478, 13877, 34299, 34485, 34642, 37941]